### 第4章 处理对象

#### 4.1 创建并使用类

可以通过编写构造器来创建一个类的实例。可以将构造器视为对象工厂。**主构造器**(primary constructor, 非正式地用其指代类)的参数定义了字段，并自动生成了访问器方法。在Scala中，不需要像Java一样需要在类中定义一个与类名相同的方法来当作构造器。

In [3]:
class Car(val year: Int) {
    private var milesDriven: Int = 0
    def miles: Int = milesDriven
    def drive(distance: Int): Unit = {
        milesDriven += distance
    }
}

val car = new Car(2015)
println(s"Car made in year ${car.year}")
println(s"Miles dirven ${car.miles}")
println("Drive for 10 miles")
car.drive(10)
println(s"Miles driven ${car.miles}")


Car made in year 2015
Miles dirven 0
Drive for 10 miles
Miles driven 10


defined class Car
car: Car = Car@1c8c4832


如果类定义没有主体，就没有必要使用大括号({})。

```scala
class CreditCard(val number: Int, var creditLimit: Int)
```

类`CreditCard`带有两个字段、一个构造器、不可变的number的getter以及可变的creditLimit的getter和setter。Scala编译器会自动把上面的代码转化为全面的类。先编译前面的代码，并运行命令`javap -private CreditCard`查看编译器所生成的代码

```scala
Compiled from "CreditCard.scala"
public class CreditCard {
  private final int number;
  private int creditLimit;
  public int number();
  public int creditLimit();  // get方法
  public void creditLimit_$eq(int); // set方法
  public CreditCard(int, int);
}
```

可以看到**编译器自动生成了`getter`和`setter`方法以及构造器**。但是`getter`和`setter`方法并不遵循JavaBean惯例。

Scala会执行主构造器中任意表达式和直接内置在类定义中的可执行语句。

In [5]:
class Construct(param: String) {
    println(s"Creating an instance of Construct with parameter $param")
}

println("Let's create an instance")
new Construct("Sample")

Let's create an instance
Creating an instance of Construct with parameter Sample


defined class Construct
res3: Construct = Construct@1c0f0515


除了主构造器，可以使用名为`this()`的方法定义**辅助构造器**(auxiliary constructor).

In [7]:
class Person(val firstName: String, val lastName: String) {
    var position: String = _ // 下划线表示相应类型的默认值，用于var，不能用于val
    println(s"Creating $toString")
    
    def this(firstName :String, lastName: String, positionHeld: String) {
        this(firstName, lastName)
        position = positionHeld
    }
    
    override def toString: String = {
        s"$firstName $lastName holds $position position"
    }
}

val john = new Person("John", "Smith", "Analyst")
println(john)
val bill = new Person("Bill", "Walker")
println(bill)

Creating John Smith holds null position
John Smith holds Analyst position
Creating Bill Walker holds null position
Bill Walker holds null position


defined class Person
john: Person = John Smith holds Analyst position
bill: Person = Bill Walker holds null position


#### 4.2 遵循JavaBean惯例

Scala编译器默认生成的访问器并不遵循JavaBean的命令规范。如果类要在Java中使用，要在相应的字段声明上标记`scala.beans.BeanProperty`注解：Scala编译器会准确可靠地生成类似于JavaBean以及Scala风格的`getter`/`setter`方法。

In [ ]:
import scala.beans.BeanProperty
class Dude(@BeanProperty val firstName: String, val lastName: String) {
    @BeanProperty var position: String = _
}

javap的输出为

```scala
public class Dude {
  private final java.lang.String firstName;
  private final java.lang.String lastName;
  private java.lang.String position;
  public java.lang.String firstName(); // scala风格 get
  public java.lang.String lastName(); // scala风格 get
  public java.lang.String position(); // scala风格 get
  public void position_$eq(java.lang.String); // scala风格 set
  public java.lang.String getFirstName();  // JavaBean风格 get
  public java.lang.String getPosition();   // JavaBean风格 get
  public void setPosition(java.lang.String); // JavaBean风格 set
  public Dude(java.lang.String, java.lang.String);
}
```

#### 4.3 类型别名

使用`type A = B`给`class B`取上`class A`的别名。

#### 4.4 扩展一个类

在Scala中使用`extends`关键字扩展基类。与Java相比，多了两个非常好的限制：其一，方法的重载必须用override关键字；其二，只有主构造器能传递参数给基类的构造器。

Scala重载必须使用`override`关键字。

In [9]:
class Vehicle(val id: Int, val year: Int) {
    override def toString = s"ID: $id Year: $year"
}

class Car(override val id: Int, override val year: Int, var fuelLevel: Int)
    extends Vehicle(id, year) {
    override def toString: String = s"${super.toString} Fuel Level: $fuelLevel"
}

val car = new Car(1, 2015, 100)
println(car)

ID: 1 Year: 2015 Fuel Level: 100


defined class Vehicle
defined class Car
car: Car = ID: 1 Year: 2015 Fuel Level: 100


因为Car中的属性id和year派生自Vehicle，通过在类Car的主构造器相应的参数前加上关键字override表明了这一点。Scala编译器不会为这两个属性生成字段，而是将这些属性的访问器方法路由到基类的相应方法。

#### 4.5 参数化类型

范型或者参数化类型有助于创建能够同时应对多种不同类型的类和函数。类型可以在编译时而不是在代码编写时确定，这样能够使代码更加扩展且类型安全。在Java中，尖括号(`<>`)被用于指定范型。在Scala中使用防括号(`[]`)来替代。

In [13]:
// 参数化的函数
def echo[T](input1: T, input2: T): Unit = 
    println(s"Got $input1 (${input1.getClass})) $input2 (${input2.getClass})")
echo("hello", "there")
echo(4, 5)
echo[Int](4, 5) //指定参数类型

Got hello (class java.lang.String)) there (class java.lang.String)
Got 4 (class java.lang.Integer)) 5 (class java.lang.Integer)
Got 4 (class java.lang.Integer)) 5 (class java.lang.Integer)


echo: [T](input1: T, input2: T)Unit


创建一个参数化类和创建参数化函数一样简单。和

In [18]:
// 参数话的类
class Message[T](val content: T) {
    override def toString: String = s"message content is $content"
    def is(value: T): Boolean = value == content
}

val message1: Message[String] = new Message("howdy") // 显式指定变量类型
val message2 = new Message(42) // 让Scala推断类型
println(message1)
println(message1.is("howdy"))
println(message1.is("hi"))
println(message2.is(22))

message content is howdy
true
false
false


defined class Message
message1: Message[String] = message content is howdy
message2: Message[Int] = message content is 42


参数化类型在实例创建的时候被指定。如果尝试输入不正确的类型，就会接收到一个严格的报错信息。

In [19]:
message1.is(22) // 编译错误：类型不匹配

<console>: 29: error: type mismatch;

#### 4.6 单例对象和伴生对象

在Scala中，创建一个单例要使用关键字`object`而不是`class`。

In [21]:
import scala.collection._

// 颜色标记器
class Marker(val color: String) {
    println(s"Creating ${this}")
    override def toString = s"marker color $color"
}

// 单例，复用预先创建好的Marker实例的单例
object MarkerFactory {
    private val markers = mutable.Map(
        "red" -> new Marker("red"),
        "blue" -> new Marker("blue"),
        "yellow" -> new Marker("yellow"))
    
    def getMarker(color: String): Marker =
        markers.getOrElseUpdate(color, new Marker(color))
}

println(MarkerFactory getMarker "blue") 
println(MarkerFactory getMarker "blue") 
println(MarkerFactory getMarker "red") 
println(MarkerFactory getMarker "red") 
println(MarkerFactory getMarker "green")

Creating marker color red
Creating marker color blue
Creating marker color yellow
marker color blue
marker color blue
marker color red
marker color red
Creating marker color green
marker color green


import scala.collection._
defined class Marker
defined object MarkerFactory


**独立对象**(stand-alone object)和任何类都没有自动的联系。例如MarkerFactory。关联到类的单例称为**伴生对象**(companion object)。 

* 伴生对象的名字和对应类的名字一致。
* 每一类都可以拥有伴生对象
* 伴生对象和相应的伴生类可以放在同一个文件中

下面使用一个伴生对象对Marker这个例子进行重写。Marker的构造器被声明为private。然而，它的伴生对象可以访问它。因此可以在伴生对象中创建Marker的实例。

In [23]:
import scala.collection._

// 伴生类
class Marker private(val color: String) {  // 主构造器标记为private
    println(s"Creating ${this}")
    override def toString = s"marker color $color"
}

object Marker {
    private val markers = mutable.Map(
        "red" -> new Marker("red"),
        "blue" -> new Marker("blue"),
        "yellow" -> new Marker("yellow"))
    
    def getMarker(color: String): Marker =
        markers.getOrElseUpdate(color, new Marker(color))
}

println(Marker getMarker "blue") 
println(Marker getMarker "blue") 
println(Marker getMarker "red") 
println(Marker getMarker "red") 
println(Marker getMarker "green")

Creating marker color red
Creating marker color blue
Creating marker color yellow
marker color blue
marker color blue
marker color red
marker color red
Creating marker color green
marker color green


import scala.collection._
defined class Marker
defined object Marker


#### 4.7 创建枚举类



#### 4.8 包对象